In [1]:
import numpy as np
from Robot import UR5Arm
from fwdkin import fwdkin
from invkin import invkin
import math

In [2]:
ex = np.array([1,0,0])
ey = np.array([0,1,0])
ez = np.array([0,0,1])
l0 = 0.061 # base to servo 1
l1 = 0.0435 # servo 1 to servo 2
l2 = 0.08285 # servo 2 to servo 3
l3 = 0.08285 # servo 3 to servo 4
l4 = 0.07385 # servo 4 to servo 5
l5 = 0.05457 # servo 5 to gripper
P01 = ( l0 + l1 ) * ez 
P12 = np.zeros (3) # translation between 1 and 2 frame in 1 frame
P23 = l2 * ex # translation between 2 and 3 frame in 2 frame
P34 = - l3 * ez # translation between 3 and 4 frame in 3 frame
P45 = np.zeros (3) # translation between 4 and 5 frame in 4 frame
P5T = -( l4 + l5 ) * ex 
print(P01,P12,P23,P34,P45,P5T)

[0.     0.     0.1045] [0. 0. 0.] [0.08285 0.      0.     ] [-0.      -0.      -0.08285] [0. 0. 0.] [-0.12842 -0.      -0.     ]


In [3]:
P = np.array([P01, P12, P23, P34, P45, P5T]).T
H = np.array([ez, -ey, -ey, -ey, -ex]).T
print(P,H)
limits = None 
dofbot = UR5Arm(P, H, limits)

[[ 0.       0.       0.08285 -0.       0.      -0.12842]
 [ 0.       0.       0.      -0.       0.      -0.     ]
 [ 0.1045   0.       0.      -0.08285  0.      -0.     ]] [[ 0  0  0  0 -1]
 [ 0 -1 -1 -1  0]
 [ 1  0  0  0  0]]


In [4]:
theta = np.array([90,90,90,90,90])[None].T*math.pi/180
print(theta)
print(theta.shape)
T = np.around(fwdkin(dofbot, theta), 4)
print(T)

[[1.57079633]
 [1.57079633]
 [1.57079633]
 [1.57079633]
 [1.57079633]]
(5, 1)
[[-0.     -0.     -1.      0.    ]
 [-0.     -1.      0.      0.    ]
 [-1.      0.     -0.      0.3986]
 [ 0.      0.      0.      1.    ]]


In [5]:
theta = np.array([0,45,135,45,135])[None].T*math.pi/180
T_0 = fwdkin(dofbot, theta)
Rd = T_0[:3, :3]
Pd = T_0[:3, -1]
print(Pd, Rd)
q0 = np.array([15,50,145,30,129])[None].T*math.pi/180
print(q0)
truth, rv = invkin(dofbot, Rd, Pd, q0)
print(rv)
print(f"before: \n{T_0}")
print(f"after: \n{np.around(fwdkin(dofbot, rv),3)}")


[0.14939045 0.         0.33674045] [[-0.70710678 -0.5        -0.5       ]
 [ 0.         -0.70710678  0.70710678]
 [-0.70710678  0.5         0.5       ]]
[[0.26179939]
 [0.87266463]
 [2.53072742]
 [0.52359878]
 [2.25147474]]
[[-0.6830127  -0.36792046 -0.630974  ]
 [-0.1830127  -0.7501044   0.63549173]
 [-0.70710678  0.54952518  0.44499672]] [0.11844033 0.03173599 0.33880039]
[-3.68502735e-05 -9.36160359e-06  9.49039457e-05 -9.44733184e-06
  1.96816290e-05 -3.20869608e-06]
Inverse Kinematics Converged after 72 iterations
[[1.31754564e-04]
 [7.85328902e-01]
 [2.35641648e+00]
 [7.85254797e-01]
 [2.35614238e+00]]
before: 
[[-0.70710678 -0.5        -0.5         0.14939045]
 [ 0.         -0.70710678  0.70710678  0.        ]
 [-0.70710678  0.5         0.5         0.33674045]
 [ 0.          0.          0.          1.        ]]
after: 
[[-7.071e-01 -4.999e-01 -5.001e-01  1.494e-01]
 [-1.000e-04 -7.071e-01  7.071e-01  0.000e+00]
 [-7.071e-01  5.000e-01  5.000e-01  3.367e-01]
 [ 0.000e+00  0.000e+

In [6]:
Rd = T[:3, :3]
Pd = T[:3, -1]
print(Rd, Pd)
# Pd = np.array([0.06568421, 0.08820059, 0.06029362])
q0 = np.random.rand(5,1)*90
# q0 = np.array([90,90,90,90,90]).reshape(5,1)
print(q0)

[[-0. -0. -1.]
 [-0. -1.  0.]
 [-1.  0. -0.]] [0.     0.     0.3986]
[[38.37669093]
 [38.92113452]
 [43.76889885]
 [66.84660928]
 [26.8163302 ]]


In [7]:
val, angles = invkin(dofbot, Rd, Pd, q0)
print(angles)

before: [[38.37669093]
 [38.92113452]
 [43.76889885]
 [66.84660928]
 [26.8163302 ]]
after: [[38.38592273]
 [38.78025049]
 [43.95188673]
 [66.78244135]
 [26.88906027]]
[[ 0.22032406 -0.61153904 -0.75991928]
 [ 0.18066324 -0.74000025  0.64788921]
 [-0.95855    -0.28003506 -0.05255724]] [0.05743274 0.0470942  0.2595402 ]
before: [[38.38592273]
 [38.78025049]
 [43.95188673]
 [66.78244135]
 [26.88906027]]
after: [[38.39290116]
 [38.66894771]
 [44.09230263]
 [66.73123969]
 [26.95535598]]
before: [[38.39290116]
 [38.66894771]
 [44.09230263]
 [66.73123969]
 [26.95535598]]
after: [[38.39819268]
 [38.56812604]
 [44.2116061 ]
 [66.6911539 ]
 [27.01552154]]
before: [[38.39819268]
 [38.56812604]
 [44.2116061 ]
 [66.6911539 ]
 [27.01552154]]
after: [[38.40221525]
 [38.47051458]
 [44.31766028]
 [66.6619542 ]
 [27.06994813]]
before: [[38.40221525]
 [38.47051458]
 [44.31766028]
 [66.6619542 ]
 [27.06994813]]
after: [[38.40528044]
 [38.37186648]
 [44.41443933]
 [66.64415316]
 [27.11906849]]
before: [[38

In [8]:
test_homo = fwdkin(dofbot, angles)
print(test_homo)
print(test_homo - T)

[[58.88123639]
 [43.11384958]
 [37.41024731]
 [66.57390065]
 [39.90381284]]
[[58.88123639]
 [43.11384958]
 [37.41024731]
 [66.57390065]
 [39.90381284]]
[[58.88123639]
 [43.11384958]
 [37.41024731]
 [66.57390065]
 [39.90381284]]
[[58.88123639]
 [43.11384958]
 [37.41024731]
 [66.57390065]
 [39.90381284]]
[[58.88123639]
 [43.11384958]
 [37.41024731]
 [66.57390065]
 [39.90381284]]
[[ 0.07697128  0.72146716 -0.68815737  0.03260562]
 [ 0.         -0.69020499 -0.7236139  -0.07561765]
 [-0.99703331  0.05569749 -0.05312596  0.07713036]
 [ 0.          0.          0.          1.        ]]
[[ 0.36066237  1.57873462 -1.1178222  -0.0043391 ]
 [ 0.         -0.24213137  0.17038277  0.017805  ]
 [-1.95594903  0.30931638 -0.18024046 -0.09173647]
 [ 0.          0.          0.          0.        ]]
